Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [4]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [5]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [6]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [7]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

1st implementation - multinomial logistic regression with SGD and L2 regularization

In [24]:
## build the computation graph

batch_size = 128
beta = 0.005

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + beta*tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [23]:
## run the computation and iterate

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 30.905212
Minibatch accuracy: 16.4%
Validation accuracy: 14.8%
Minibatch loss at step 500: 1.821579
Minibatch accuracy: 79.7%
Validation accuracy: 79.8%
Minibatch loss at step 1000: 0.993097
Minibatch accuracy: 77.3%
Validation accuracy: 80.6%
Minibatch loss at step 1500: 0.792733
Minibatch accuracy: 84.4%
Validation accuracy: 81.2%
Minibatch loss at step 2000: 0.592993
Minibatch accuracy: 85.2%
Validation accuracy: 81.9%
Minibatch loss at step 2500: 0.705144
Minibatch accuracy: 80.5%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 0.843064
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
Test accuracy: 88.4%


2nd implementation - 1-hidden layer neural network with SGD and L2 regularization

In [37]:
## build the computation graph

batch_size = 128
hidden_nodes = 1024
beta = 0.005

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes]))
    
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1), weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
        beta*(tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [38]:
## run the computation and iterate

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1992.312012
Minibatch accuracy: 7.8%
Validation accuracy: 20.9%
Minibatch loss at step 500: 127.051697
Minibatch accuracy: 81.2%
Validation accuracy: 82.4%
Minibatch loss at step 1000: 11.032025
Minibatch accuracy: 81.2%
Validation accuracy: 85.0%
Minibatch loss at step 1500: 1.486217
Minibatch accuracy: 83.6%
Validation accuracy: 85.3%
Minibatch loss at step 2000: 0.643919
Minibatch accuracy: 89.1%
Validation accuracy: 85.1%
Minibatch loss at step 2500: 0.677423
Minibatch accuracy: 82.8%
Validation accuracy: 85.0%
Minibatch loss at step 3000: 0.715979
Minibatch accuracy: 82.8%
Validation accuracy: 85.8%
Test accuracy: 91.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [45]:
## run the computation and iterate
# restrict training data to 5 batches by setting limit on offset

num_steps = 3001
training_batches = 5

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (training_batches * (batch_size - 1))
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Offset: %d" % offset)
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1908.558228
Minibatch accuracy: 7.0%
Validation accuracy: 38.9%
Offset: 0
Minibatch loss at step 500: 128.664078
Minibatch accuracy: 100.0%
Validation accuracy: 76.6%
Offset: 500
Minibatch loss at step 1000: 10.642681
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Offset: 365
Minibatch loss at step 1500: 1.081248
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Offset: 230
Minibatch loss at step 2000: 0.289470
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Offset: 95
Minibatch loss at step 2500: 0.232001
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Offset: 595
Minibatch loss at step 3000: 0.200875
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Offset: 460
Test accuracy: 86.0%


As expected, using just a handfull of batches of training data (in this case 640 images) results in a very high accuracy on the training set, but reduced accuracy on the validation and test sets. In other words, extreme overfitting.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

1-hidden layer neural network with SGD, L2 regularization and dropout on the hidden layer

In [116]:
## build the computation graph

batch_size = 128
hidden_nodes = 1024
beta = 0.005

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes]))
    
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Model
  def model(data):
    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(data, weights_1) + biases_1), keep_prob=0.5)
    return tf.matmul(hidden, weights_2) + biases_2

  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
        beta*(tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)))

  # use learning rate decay
  global_step = tf.Variable(0)
  starter_learning_rate = 0.5
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 25, 0.96)
  
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)  

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [117]:
## run the computation and iterate

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2101.015137
Minibatch accuracy: 8.6%
Validation accuracy: 27.9%
Minibatch loss at step 500: 283.589325
Minibatch accuracy: 78.1%
Validation accuracy: 81.9%
Minibatch loss at step 1000: 131.383148
Minibatch accuracy: 78.1%
Validation accuracy: 83.7%
Minibatch loss at step 1500: 94.165207
Minibatch accuracy: 78.1%
Validation accuracy: 84.6%
Minibatch loss at step 2000: 81.044479
Minibatch accuracy: 84.4%
Validation accuracy: 85.2%
Minibatch loss at step 2500: 76.146912
Minibatch accuracy: 81.2%
Validation accuracy: 85.3%
Minibatch loss at step 3000: 73.926956
Minibatch accuracy: 82.8%
Validation accuracy: 85.2%
Test accuracy: 91.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [171]:
## build the computation graph

batch_size = 128
hidden_nodes_1 = 1024
hidden_nodes_2 = 300
hidden_nodes_3 = 50
beta = 1e-5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_1], stddev=np.sqrt(2.0/784)))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes_1]))

  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_1, hidden_nodes_2], stddev=np.sqrt(2.0/1024)))
  biases_2 = tf.Variable(tf.zeros([hidden_nodes_2]))
    
  weights_3 = tf.Variable(
    tf.truncated_normal([hidden_nodes_2, hidden_nodes_3], stddev=np.sqrt(2.0/300)))
  biases_3 = tf.Variable(tf.zeros([hidden_nodes_3]))

  weights_4 = tf.Variable(
    tf.truncated_normal([hidden_nodes_3, num_labels], stddev=np.sqrt(2.0/50)))
  biases_4 = tf.Variable(tf.zeros([num_labels]))

  # Model
  def model(data):
    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(data, weights_1) + biases_1), keep_prob=0.5)
    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden, weights_2) + biases_2), keep_prob=0.5)
    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden, weights_3) + biases_3), keep_prob=0.5)
    return tf.matmul(hidden, weights_4) + biases_4

  # Training computation.
  logits = model(tf_train_dataset)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
        beta*(tf.nn.l2_loss(weights_1) + 
              tf.nn.l2_loss(weights_2) +
              tf.nn.l2_loss(weights_3) +
              tf.nn.l2_loss(weights_4)))
  
  # use learning rate decay
  global_step = tf.Variable(0)
  starter_learning_rate = 0.5
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 2500, 0.7)
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)  
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [172]:
## run the computation and iterate

num_steps = 15001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.846983
Minibatch accuracy: 9.4%
Validation accuracy: 13.3%
Minibatch loss at step 500: 0.615127
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 0.731530
Minibatch accuracy: 75.8%
Validation accuracy: 82.5%
Minibatch loss at step 1500: 0.489955
Minibatch accuracy: 85.2%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 0.417118
Minibatch accuracy: 89.1%
Validation accuracy: 83.5%
Minibatch loss at step 2500: 0.580329
Minibatch accuracy: 81.2%
Validation accuracy: 84.2%
Minibatch loss at step 3000: 0.574255
Minibatch accuracy: 80.5%
Validation accuracy: 84.8%
Minibatch loss at step 3500: 0.448548
Minibatch accuracy: 89.1%
Validation accuracy: 84.8%
Minibatch loss at step 4000: 0.643539
Minibatch accuracy: 82.8%
Validation accuracy: 85.4%
Minibatch loss at step 4500: 0.420101
Minibatch accuracy: 89.1%
Validation accuracy: 85.4%
Minibatch loss at step 5000: 0.418090
Minibatch accuracy: 91.4%
Validation accuracy